In [1]:
!unzip /content/face.evoLVe-master.zip

Archive:  /content/face.evoLVe-master.zip
a9897bda52bdbb8d7c2fe28f1e21827dfd69d14e
   creating: face.evoLVe-master/
  inflating: face.evoLVe-master/.gitignore  
  inflating: face.evoLVe-master/LICENSE  
  inflating: face.evoLVe-master/README.md  
   creating: face.evoLVe-master/applications/
   creating: face.evoLVe-master/applications/align/
 extracting: face.evoLVe-master/applications/align/__init__.py  
  inflating: face.evoLVe-master/applications/align/align_trans.py  
  inflating: face.evoLVe-master/applications/align/box_utils.py  
  inflating: face.evoLVe-master/applications/align/detector.py  
  inflating: face.evoLVe-master/applications/align/face_align.py  
  inflating: face.evoLVe-master/applications/align/face_resize.py  
  inflating: face.evoLVe-master/applications/align/first_stage.py  
  inflating: face.evoLVe-master/applications/align/get_nets.py  
  inflating: face.evoLVe-master/applications/align/matlab_cp2tform.py  
  inflating: face.evoLVe-master/applications/align/

This command will extract all the files and folders contained within `face.evoLVe.zip` into the current directory. If you want to extract to a specific directory, you can use the `-d` option followed by the directory path.

In [2]:
!unzip dataset_augmented.zip

Archive:  dataset_augmented.zip
   creating: dataset_augmented/
   creating: dataset_augmented/Brian Alexander/
  inflating: dataset_augmented/Brian Alexander/aug_0_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Alexander/aug_1_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Alexander/aug_2_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Alexander/aug_3_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Alexander/aug_4_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Alexander/aug_5_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Alexander/aug_6_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Alexander/aug_7_WhatsApp Image 2025-09-21 at 11.14.19_355b0534.jpg  
  inflating: dataset_augmented/Brian Ale

In [4]:
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2

In [5]:
!pip install scikit-learn matplotlib onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


In [1]:
!pip install deepface

  Using cached deepface-0.0.96-py3-none-any.whl.metadata (35 kB)
  Using cached flask_cors-6.0.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached mtcnn-1.0.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached retina_face-0.0.17-py3-none-any.whl.metadata (10 kB)
  Using cached fire-0.7.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached lightphe-0.0.19-py3-none-any.whl.metadata (13 kB)
  Using cached lightecc-0.0.3-py3-none-any.whl.metadata (14 kB)
  Using cached lz4-4.4.5-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached numpy-2.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytest-7.1.2-py3-none-any.whl.metadata (7.8 kB)
  Using cached py-1.11.0-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached tomli-2.3.

In [1]:
import deepface
deepface.__version__

'0.0.96'

In [2]:
!pip install deepface==0.0.96 scikit-learn tqdm

In [3]:
from deepface import DeepFace
import os
import glob
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

25-12-10 09:20:40 - Directory /root/.deepface has been created
25-12-10 09:20:40 - Directory /root/.deepface/weights has been created


In [4]:
DATASET = "/content/dataset_augmented"

classes = os.listdir(DATASET)
print("Detected classes:", classes)

Detected classes: ['Brian Alexander', 'Melvern', 'Jason']


In [5]:
embeddings = []
labels = []

for person in classes:
    folder = os.path.join(DATASET, person)
    image_files = glob.glob(folder + "/*.jpg")

    print(f"Processing {person} : {len(image_files)} images")

    for img_path in tqdm(image_files):

        try:
            rep = DeepFace.represent(
                img_path=img_path,
                model_name="ArcFace",
                enforce_detection=False
            )

            # DeepFace returns a list → get first dict → extract embedding vector
            emb = np.array(rep[0]["embedding"])

            embeddings.append(emb)
            labels.append(person)

        except Exception as e:
            print("Skipped:", img_path, "| Reason:", e)

Processing Brian Alexander : 10 images


  0%|          | 0/10 [00:00<?, ?it/s]

25-12-10 09:20:46 - 🔗 arcface_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5 to /root/.deepface/weights/arcface_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5

  0%|          | 0.00/137M [00:00<?, ?B/s]
 14%|█▍        | 19.4M/137M [00:00<00:00, 193MB/s]
 29%|██▊       | 39.3M/137M [00:00<00:00, 194MB/s]
 43%|████▎     | 59.2M/137M [00:00<00:00, 164MB/s]
 56%|█████▌    | 76.5M/137M [00:00<00:00, 143MB/s]
 69%|██████▉   | 94.4M/137M [00:00<00:00, 153MB/s]
 82%|████████▏ | 113M/137M [00:00<00:00, 161MB/s] 
100%|██████████| 137M/137M [00:00<00:00, 139MB/s]
100%|██████████| 10/10 [00:33<00:00,  3.36s/it]


Processing Melvern : 44 images


100%|██████████| 44/44 [01:09<00:00,  1.57s/it]


Processing Jason : 10 images


100%|██████████| 10/10 [03:00<00:00, 18.01s/it]


In [6]:
X = np.array(embeddings)
y = np.array(labels)

print("Embedding matrix:", X.shape)
print("Labels:", y.shape)

Embedding matrix: (64, 512)
Labels: (64,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [8]:
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [9]:
y_pred = svm.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy:", acc)

Test Accuracy: 0.9230769230769231


In [10]:
import joblib

joblib.dump(svm, "face_classifier_svm.pkl")
np.save("embeddings_labels.npy", y)

In [11]:
def predict_face(img_path):

    rep = DeepFace.represent(
        img_path,
        model_name="ArcFace",
        enforce_detection=False
    )

    if isinstance(rep, list):
        emb = np.array(rep[0]["embedding"])
    elif isinstance(rep, dict):
        emb = np.array(rep["embedding"])
    else:
        raise ValueError("Unknown embedding format:", rep)

    emb = emb.reshape(1, -1)

    pred = svm.predict(emb)[0]
    prob = svm.predict_proba(emb)[0].max()

    return pred, prob

In [12]:
image_to_predict = "/content/WhatsApp Image 2025-12-10 at 16.28.02_cb4337b5.jpg"
predicted_class, confidence_score = predict_face(image_to_predict)
print(f"Predicted class: {predicted_class}, Confidence: {confidence_score:.4f}")

Predicted class: Brian Alexander, Confidence: 0.6651
